<a href="https://colab.research.google.com/github/abyanjan/Natural-language-Processing-NLP-/blob/master/Topic_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modelling 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition
from sklearn.model_selection import train_test_split

import re
import nltk
from nltk.stem.porter import PorterStemmer


In [2]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
# Data
df = pd.read_csv('https://github.com/srivatsan88/YouTubeLI/blob/master/dataset/consumer_compliants.zip?raw=true',compression='zip',sep=',',quotechar='"')

In [ ]:
df.head()

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,4/3/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Fraudulent loan,This auto loan was opened on XX/XX/2020 in XXX...,Company has responded to the consumer and the ...,TRUIST FINANCIAL CORPORATION,PA,None,None,Consent provided,Web,4/3/2020,Closed with explanation,Yes,NaN,3591341
1,3/12/2020,Debt collection,Payday loan debt,Attempts to collect debt not owed,Debt is not yours,In XXXX of 2019 I noticed a debt for {$620.00}...,None,CURO Intermediate Holdings,CO,806XX,None,Consent provided,Web,3/12/2020,Closed with explanation,Yes,NaN,3564184
2,2/6/2020,Vehicle loan or lease,Loan,Getting a loan or lease,Credit denial,"As stated from Capital One, XXXX XX/XX/XXXX an...",None,CAPITAL ONE FINANCIAL CORPORATION,OH,430XX,None,Consent provided,Web,2/6/2020,Closed with explanation,Yes,NaN,3521949
3,3/6/2020,Checking or savings account,Savings account,Managing an account,Banking errors,"Please see CFPB case XXXX. \n\nCapital One, in...",None,CAPITAL ONE FINANCIAL CORPORATION,CA,None,None,Consent provided,Web,3/6/2020,Closed with explanation,Yes,NaN,3556237
4,2/14/2020,Debt collection,Medical debt,Attempts to collect debt not owed,Debt is not yours,This debt was incurred due to medical malpract...,Company believes it acted appropriately as aut...,"Merchants and Professional Bureau, Inc.",OH,432XX,None,Consent provided,Web,2/14/2020,Closed with explanation,Yes,NaN,3531704


Data is related to customer complaints made by customers in different services related to the banks.

In [ ]:
df['Product'].value_counts()

Debt collection                21772
Credit card or prepaid card    13193
Mortgage                        9799
Checking or savings account     7003
Student loan                    2950
Vehicle loan or lease           2736
Name: Product, dtype: int64

In [ ]:
df['Company'].value_counts()

CITIBANK, N.A.                                                 3226
CAPITAL ONE FINANCIAL CORPORATION                              2711
BANK OF AMERICA, NATIONAL ASSOCIATION                          2580
JPMORGAN CHASE & CO.                                           2409
WELLS FARGO & COMPANY                                          2001
                                                               ... 
Rhodes, Kelly Assoc Inc                                           1
Miller and Steeno, P.C.                                           1
RABOBANK, NATIONAL ASSOCIATION                                    1
KMD Partners, LLC                                                 1
Citizens Savings & Loan Corporation, Chattanooga, TN Branch       1
Name: Company, Length: 2197, dtype: int64

In [4]:
# selecting the columns and creating a new data
complaint_df = df[['Consumer complaint narrative','Product','Company']].rename(columns = {'Consumer complaint narrative':'complaints'})

In [5]:
pd.set_option('display.max_colwidth', -1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  """Entry point for launching an IPython kernel.


In [ ]:
complaint_df.head()

,complaints,Product,Company
0,"This auto loan was opened on XX/XX/2020 in XXXX, NC with BB & T in my name. I have NEVER been to North Carolina and I have NEVER been a resident. I have filed a dispute twice through my credit bureaus but both times BB & T has claimed that this is an accurate loan. Which I wasn't aware of until today. I have tried to contact BB & T multiple times but I have never gotten through to a live person. I do n't drive and I have never owned a car before. I didn't have any knowledge of this account until I checked XXXXXXXX XXXX and noticed it. I've tried twice to dispute it. Additionally I never received any bills or information about this account. This is my last resort in trying to remove this fraudulent loan off of my account.",Vehicle loan or lease,TRUIST FINANCIAL CORPORATION
1,"In XXXX of 2019 I noticed a debt for {$620.00} on my credit which i believed was mine I thought speedy cash had bought one of my old debts and sold it to XXXX XXXX XXXX XXXX. I contacted XXXX XXXX XXXX XXXX and after several attempts of giving my full name, nothing came up in their system. I gave my social and the rep said the account popped up but DID NOT tell me that the account was under someone elses name and continued to let me make a payment. The payment was for {$120.00}. Confirmation number-XXXX. After realizing it was not my account, I called back to get my money back and inform them of the mistake. I was told i needed to mail them an FTC report and dispute letter to get my money back. I completed all of this and when i called again they said they transferred the account back to speedy cash for fraud review and I would need to contact them. After contacting them i was again told that i can not get my money back. The issue im having is this representative at XXXX XXXX played blind to obvious fraud and let an innocent person make a payment on someone elses debt and i want my money back.",Debt collection,CURO Intermediate Holdings
2,"As stated from Capital One, XXXX XX/XX/XXXX and XXXX 2018, My wife and I went to several car dealerships to request for a car loan to get a used car. However, according to their credit requirements unfortunately my credit score was insufficient for the car loan approval at that time. It seemed as though they pulled my credit report multiple times.",Vehicle loan or lease,CAPITAL ONE FINANCIAL CORPORATION
3,"Please see CFPB case XXXX. \n\nCapital One, in the letter they provided ( and attached to that case as their response ) said this : "" The funds were reversed and sent back to XXXX XXXX XXXX on XX/XX/XXXX ''. \n\nXXXX XXXX XXXX ( now XXXX XXXX ) has not received these funds. Staff at XXXX XXXX - and also staff at the account-holder 's business - have looked for return of my money ( {$650.00} ) and find nothing. \n\nCapital One needs to document - actually prove - they returned the funds, as stated in their letter. Capital One must provide electronic information, if the return was made that way, or document the paper check they sent back to XXXX XXXX. \n\nI've left 3 messages about this problem for the person who signed the letter ( XXXX ) from Capital One. I have received no call-backs. \n\nSummary : Capital One said they returned my money on XX/XX/XXXX : they did not. If they continue claim they did, then they need to prove that.",Checking or savings account,CAPITAL ONE FINANCIAL CORPORATION
4,"This debt was incurred due to medical malpractice ( XXXX XXXX XXXX, XXXX, TX ). I asked the doctor to turn over my claim to his malpractice insurance company. This has cost me thousands of dollars to XXXX XXXX XXXX. I am still trying to collect damages from this doctor. He never responded and turned over me to collections Merchants and Professional Collection Bureau , Inc. I sent them a letter describing exactly this issue and instead of not contacting me and verifying my debt they start reporting this debt to the credit reporting agencies. They never verified the debt, like I asked and they never

**Train Test Split**

In [6]:
X_train, X_test = train_test_split(complaint_df, test_size = 0.5, random_state = 11)

In [ ]:
X_train.Product.value_counts()

Debt collection                10882
Credit card or prepaid card    6551 
Mortgage                       4873 
Checking or savings account    3527 
Student loan                   1512 
Vehicle loan or lease          1381 
Name: Product, dtype: int64

**Data Processing**

In [ ]:
# initialize stemming
#stemmer = PorterStemmer()

In [7]:
import spacy

In [11]:
nlp = spacy.load("en_core_web_sm")

In [31]:
# word tokenization function
def tokenize(text):
  tokens = [word for word in nltk.word_tokenize(text) if (len(word) > 3 and len(word.strip('Xx/')) > 2 and len(re.sub('\d+','', word.strip('Xx/'))) > 2)]
  # lemmatization with spacy
  doc = nlp(" ".join(tokens))
  #stems = [stemmer.stem(word) for word in tokens]
  lemma_words = [word.lemma_ for word in doc]
  return lemma_words

In [34]:
# count vectorizer
vectorizer = CountVectorizer(tokenizer=tokenize, stop_words='english', max_df=0.75, min_df=50, max_features=10000)

In [35]:
train_vectors = vectorizer.fit_transform(X_train['complaints'])

/usr/local/lib/python3.6/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['-PRON-', 'make'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [36]:
train_vectors.A

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:
vectorizer.get_feature_names()[1:20]

['*',
 '-',
 '.',
 '..',
 '/',
 'aadvantage',
 'abide',
 'ability',
 'able',
 'abruptly',
 'absolute',
 'absolutely',
 'absurd',
 'abuse',
 'abusive',
 'accelerate',
 'accept',
 'acceptable',
 'acceptance']

## Topic Modelling with LDA (Latent Dirichlet Allocation)




In [38]:
lda = decomposition.LatentDirichletAllocation(n_components=6, max_iter=10,learning_method='online', learning_offset=50,n_jobs=-1, random_state=111)

In [39]:
# probability of a document belonging to the 6 topics
topic_weights = lda.fit_transform(train_vectors)
topic_weights

array([[0.00542894, 0.64761597, 0.03868751, 0.00541285, 0.29743759,
        0.00541714],
       [0.38216682, 0.00184237, 0.00184893, 0.00184933, 0.00183886,
        0.61045369],
       [0.47790268, 0.25308094, 0.00236216, 0.06515623, 0.00238013,
        0.19911786],
       ...,
       [0.00371151, 0.00371517, 0.19748551, 0.00371815, 0.00372238,
        0.78764727],
       [0.00671001, 0.00671333, 0.0066974 , 0.57648036, 0.39671026,
        0.00668864],
       [0.00351633, 0.00350472, 0.00350562, 0.27788607, 0.53340237,
        0.17818488]])

In [40]:
# lda components
components = lda.components_

In [41]:
# taking top 15 words from each topic
num_words = 15

# vocabulary of all the words
vocab = np.array(vectorizer.get_feature_names())

# function to extract top words
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]

topic_words = ([top_words(t) for t in components])
topic_words = [" ".join(t) for t in topic_words]

In [42]:
topic_words

['account card tell say time credit phone number ask receive close send chase bank speak',
 'bank charge account check card transaction dispute fund receive claim purchase deposit make fraud america',
 'payment pay make month late receive mortgage account balance time send insurance tell statement loan',
 'debt collection letter send company provide receive request information state contact collect claim dispute proof',
 'credit report account remove information company debt identity theft collection score open dispute balance reporting',
 'loan mortgage home - year time property work document rate make tell income modification state']

**Adding the topics to the data**

In [43]:
colnames = ['Topic'+str(i) for i in range(lda.n_components)]
documents = ['Doc'+str(i) for i in range(len(X_train.complaints))]

df_doc_topic = pd.DataFrame(np.round(topic_weights,2), columns=colnames, index=documents)

top_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['Dominant_topic'] = top_topic

In [44]:
df_doc_topic

,Topic0,Topic1,Topic2,Topic3,Topic4,Topic5,Dominant_topic
Doc0,0.01,0.65,0.04,0.01,0.30,0.01,1
Doc1,0.38,0.00,0.00,0.00,0.00,0.61,5
Doc2,0.48,0.25,0.00,0.07,0.00,0.20,0
Doc3,0.06,0.00,0.92,0.00,0.00,0.00,2
Doc4,0.00,0.00,0.00,0.28,0.70,0.00,4
...,...,...,...,...,...,...,...
Doc28721,0.26,0.68,0.00,0.05,0.00,0.00,1
Doc28722,0.62,0.19,0.07,0.00,0.11,0.00,0
Doc28723,0.00,0.00,0.20,0.00,0.00,0.79,5
Doc28724,0.01,0.01,0.01,0.58,0.40,0.01,3


**Applying the topic modeling to test data**

In [ ]:
# testing for first 5 rows
test_topic_weights = lda.transform(vectorizer.transform(X_test.complaints[:5]))

In [ ]:
colnames = ['Topic'+str(i) for i in range(lda.n_components)]
documents = ['Doc'+str(i) for i in range(len(X_test.complaints[:5]))]

df_doc_topic = pd.DataFrame(np.round(test_topic_weights,2), columns=colnames, index=documents)

top_topic = np.argmax(df_doc_topic.values, axis=1)
df_doc_topic['Dominant_topic'] = top_topic

## Topic Modelling with Truncated SVD

In [45]:
from sklearn.decomposition import TruncatedSVD

In [46]:
trunc_SVD = TruncatedSVD(n_components=6, n_iter=10, random_state=12)

In [47]:
trunc_SVD.fit(train_vectors)

TruncatedSVD(algorithm='randomized', n_components=6, n_iter=10, random_state=12,
             tol=0.0)

In [65]:
components = trunc_SVD.components_

**Topics**

In [66]:
# taking top 15 words from each topic
num_words = 15

# vocabulary of all the words
vocab = np.array(vectorizer.get_feature_names())

# function to extract top words
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]

topic_words = ([top_words(t) for t in components])
topic_words = [" ".join(t) for t in topic_words]

In [67]:
topic_words

['account payment credit loan receive make bank tell card time send say information charge mortgage',
 'loan payment mortgage make home month property modification pay insurance - escrow document servicer year',
 'debt credit report collection information provide company request dispute letter loan consumer agency state file',
 'account bank loan check mortgage information deposit fund provide money state document - request property',
 'card loan tell charge say bank receive email purchase ask time home fraud chase phone',
 'loan credit account card report balance mortgage close student servicer open rate - application year']

## Topic Modelling with NMF(Non negative Matrix Factorization)

In [68]:
from sklearn.decomposition import NMF

In [69]:
model = NMF(n_components=6, random_state=12)

In [70]:
W = model.fit_transform(train_vectors)

In [72]:
components = model.components_

**Topics**

In [73]:
# taking top 15 words from each topic
num_words = 15

# vocabulary of all the words
vocab = np.array(vectorizer.get_feature_names())

# function to extract top words
top_words = lambda t: [vocab[i] for i in np.argsort(t)[:-num_words-1:-1]]

topic_words = ([top_words(t) for t in components])
topic_words = [" ".join(t) for t in topic_words]

In [74]:
topic_words

['account bank check open close money deposit fund balance transfer fee information branch customer business',
 'payment make month late pay mortgage time balance statement credit receive monthly apply date tell',
 'debt credit report collection information company provide request consumer dispute letter send state agency creditor',
 'card credit charge purchase balance report fraud make use dispute transaction capital citi time statement',
 'tell receive say send time bank ask email letter phone state speak contact number need',
 'loan mortgage - home property document request servicer year modification / rate provide date student']